*  This program compares the performance 
*                             of the 5 investement rules/strategies 
*  when applies to the 5 industry portfolios  
*  For simplicy, no constraints are imposed. 

In [1]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import scipy.special as sc                # to compute the beta function 


# Load the data 

df = pd.read_excel('Factors_July26_July11.xlsx')        # It has 5 columns:  date, mkt, size, b/m, riskree rate
                                                    # downloaded from Ken French's website
df2 = pd.read_excel('Indu5_July26_July11.xlsx')       # return on 5 industry portfolios from Ken French's web 

mkt = df.loc[:,"mkt"]/100                       # Mkt excess return;  divided by 100 due to data are in %
rf = df.loc[:,"rate"]/100

R1 = df2.loc[:,'Indu1' : 'Indu5']/100        # extract the 5 industry returns, R1 is T by 5. 

R1 = np.array(R1)                         # convert list to array to apply np.functions
rf = np.array(rf)                         # convert list to array to apply np.functions
T = len(df)                               # The number of obs
Re = np.ones((T,5))                           # creat storage for excess returns

for i in range(5):
    Re[:,i] = R1[:,i] - rf                # the excess return:  each indu substracts riskfree rate, Re[:,i]-rf 

T0 = 120                 # set aside data for the initial estimation

print(' Total sample size \n')
print(T)
print(' Sample size used to start estimation \n')
print(T0)

 Total sample size 

1021
 Sample size used to start estimation 

120


In [6]:
# Compute all the 5 portfolio weights 

TT = T - T0              #  from T0, T0+1, ..., to T-1
                         #  No need for T as we will not know its return

                         # declare storage for the 4 portfolios (though not necessary, but clearer)
                                      #          each is 5-vector over time
w1 = np.ones((5,TT))
w2 = np.ones((5,TT))
w3 = np.ones((5,TT))
w4 = np.ones((5,TT))
w5 = np.ones((5,TT))

# Begin with the first 4 portfolio weights, which are simpler


                    # 1):  1/N rule over T0, T0+1, ..., T-1
N = 5
w1 = np.ones((5,TT)) / N       

                    # 2):  The other 3 rules

gamma = 3     
ones5 = np.ones((5,1))            # a 5-vector of ones, to be used later

for t in range(T0,T):
    RR = np.ones((t,5))           # to store the data from 1 to t, t by 5 matrix 
    for i in range (t):           # get available data up to t
        RR[i,:] = Re[i,:] 
    mu = np.mean(RR, axis = 0)          # the mean taking each column of the matrix 
    mu = mu.T                           # make it a column vector
    V = np.cov(RR.T)                        # the covariance estimate 
    VI = np.linalg.inv(V)                     #  Inverse of V
    tt = t-T0
    w2[:,tt] = (1/gamma)*np.matmul(VI, mu)      ##  store the plug-in weights
    A = np.matmul(VI, ones5)
    b = np.matmul(ones5.T, A)
    w3C = A / b                                 ##  5 by 1, GMV weights
    for j in range(5):
        w3[j,tt] = w3C[j]                      # store GMV to w3
                                 ## Now the correlation shrinkage
    CorrV = np.corrcoef(RR.T)           # compute the correlation of the returns
    CorrV = 0.5*CorrV + 0.5*np.identity(5)     # the shrinkage
    VV = V                                  # to store the new covariane matrix 
    for i in range(5):                     # compute the covariance matrix after shrinkage
        for j in range (5):
            if j != i:
                VV[i,j] = CorrV[i,j]*np.sqrt(V[i,i])*np.sqrt(V[j,j])
    VVI = np.linalg.inv(VV)              
    tt = t-T0
    w4[:,tt] = (1/gamma)*np.matmul(VVI, mu)      ##  store the shrinakge weights
                                 # Now the last and complex weights 
    B = np.matmul(VI, mu) 
    theta2 = np.matmul(mu.T, B)             # sample Sharpe ratio squared
    x = theta2 / (1+ theta2)                # get parameters for incomplete beta function
    a = N/2.0
    b = (t-N) / 2.0
    ibeta = sc.betainc(a, b,x)*sc.beta(a, b)   # b/c sc.betainc defined with 1/sc.beta(a, b) coeff
    c = ((t-N-2)*theta2 - N)/t
    d = 2*np.power(theta2,N/2.0)*np.power(1+theta2,-(t-2)/2.0)
    theta2a = c + d / (t*ibeta)
    c1 = (t-2)*(t-N-2) / ( (t-N-1)*(t-N-4) )
    A = np.matmul(V, ones5)                 # variables in pi1 and pi2
    B = np.matmul(ones5.T, A) 
    C = np.matmul(ones5.T, mu) / N
    pi1 = B / (N*N) - (2*C)/gamma + theta2a / (gamma*gamma)
    pi2 = (c1-1)*theta2a / (gamma*gamma) + c1*N / (t*gamma*gamma)
    if pi1 < 0:                            # sets pi1 to be 0 or positive
        pi1 = 0
    delta = pi1 / (pi1+pi2)
    w2C=((t-N-2)/t)*w2[:,tt]                # the adjusted plug-in rule
    w2C.shape = (5,1)                       # make sure the next Equ gets 5 by 1 vector
    w5C = (1-delta)*ones5/N +  delta*w2C     # the combination weights 
                    # (t-N-2)/t is due to using the adjusted plug-in
    for j in range(5):
        w5[j,tt] = w5C[j]                      # store the weights as w5

print('The first 1/N  at T0 \n')
print(w1[:,0])
print('The Plug-in weights  \n')
print(w2[:,0])
print('The GMV   \n')
print(w3[:,0])
print('The Correlation Shrinkage  \n')
print(w4[:,0])
print('The Combination   \n')
print(w5[:,0])

The first 1/N  at T0 

[0.2 0.2 0.2 0.2 0.2]
The Plug-in weights  

[ 0.81871482 -1.07711541  1.44382641  0.12164157 -0.63231407]
The GMV   

[ 0.83870023 -1.02381427  1.35383763  0.1888985  -0.35762208]
The Correlation Shrinkage  

[ 0.19038552  0.0430349   0.28912913  0.15101204 -0.11387872]
The Combination   

[ 0.33939188 -0.09645217  0.48310262  0.17913743  0.00580594]


In [9]:
# Compute the Sharpe Ratios and Accu Return of the 5 rules and Mkt
        
       # write a function to do it to save space or to be cleaer

def get_SRW(w):
    Port=np.ones((TT,1))      #   to store the returns on the portfolio
    ExPort = np.ones((TT,1))      # to store excess return
    for t in range(TT):
        w5 = w[:,t]
        ExPort[t] = np.dot(w5,Re[T0+t])      # excess return of the portfolio
        Port[t] = ExPort[t] + rf[T0+t]       # add back r_f to get the raw return
    muP = ExPort.mean()
    sig2P = ExPort.var()
    sigP = np.sqrt(sig2P)
    SharpeP = np.sqrt(12)*muP/sigP        # annualize the Sharp Ratio
    CC_Port = np.ones((TT,1))          # to store the accumulative returns 
    CC_Port[0] = 1+Port[0]  
    for t in range(TT-1):
        CC_Port[t+1] = CC_Port[t]*(1+Port[t+1])
    c = CC_Port[TT-1]
    return SharpeP, c

sr1, c1 = get_SRW(w1)
sr2, c2 = get_SRW(w2)
sr3, c3 = get_SRW(w3)
sr4, c4 = get_SRW(w4)
sr5, c5 = get_SRW(w5)

# For the mkt
                                     # Sharpe ratio of the Market 
                                     #  over  T0+1, ..., T 
mkt1 = df.loc[T0:T,"mkt"]/100       # Note r_{T0+1} is stored as r[T0]
mu = mkt1.mean()                      
sig2 = mkt1.var()                       
sigma = np.sqrt(sig2)                  
SharpeM = np.sqrt(12)*mu/sigma      # annualized      

# Get the accu return for the mkt

CC = np.ones((TT,1))          # to store the accumulative returns 
                           # raw return by adding rf back
mkt2 = (df.loc[T0:T,"mkt"] + df.loc[T0:T,"rate"])/100  

print('The Mkt raw returns \n')
print (mkt2.head()) 

mkt2 = np.array(mkt2)       # the index will start from 0

CC[0] = 1 + mkt2[0]                 # initial accu return

for t in range(TT-1):
    CC[t+1] = CC[t]*(1+mkt2[t+1])
    
print('The Annualized Sharpe ratios of the Opt Port and Mkt \n')
print('          {0:.4f}   {1:.4f}  \n'.format(sr1, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr2, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr3, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr4, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr5, SharpeM)) 

print('Terminal wealth of the 5 rules  \n')
print(c1)  
print(c2)  
print(c3)  
print(c4) 
print(c5) 
print('Terminal wealth of 100% invested in Mkt  \n')
print(CC[TT-1])  


The Mkt raw returns 

120    0.0646
121    0.0108
122    0.0143
123    0.0691
124    0.0333
dtype: float64
The Annualized Sharpe ratios of the Opt Port and Mkt 

          0.5127   0.4610  

          0.4231   0.4610  

          0.5129   0.4610  

          0.4895   0.4610  

          0.5120   0.4610  

Terminal wealth of the 5 rules  

[2568.30219166]
[841.34668186]
[2529.46767975]
[3093.26014307]
[2100.07994649]
Terminal wealth of 100% invested in Mkt  

[1565.54242462]


In [ ]:
prin